# Day 1 실습 7: Gradio로 모델 배포하기

## 🎯 이 노트북의 목적

6번에서 학습한 LoRA 모델을 **Gradio UI**로 만들고 **HuggingFace Spaces**에 배포합니다.

### 배포 대상 (예시)

강사 데모:
- ✅ **감정 분류 모델** (6번에서 학습)
- ✅ **영어 QA 모델** (6번에서 학습)

학생 여러분:
- 🔧 **TODO만 채워서** 본인 모델 배포
- 🚀 **여러 모델 테스트** 가능

### 실습 흐름

1. Gradio 인터페이스 만들기 (로컬 테스트)
2. 6번에서 학습한 모델로 테스트
3. LoRA 어댑터를 HF Hub에 업로드
4. HF Spaces 배포용 파일 생성
5. HF Spaces에 업로드
6. 공개 링크로 공유

### 💡 Gradio란?

- Python으로 **3줄**이면 UI 만들 수 있는 라이브러리
- ML 모델 데모에 최적화
- HuggingFace Spaces와 완벽 호환

## 1. 패키지 설치

Gradio와 HuggingFace Hub 라이브러리를 설치합니다.

**⚠️ 중요: Colab에서는 설치 후 런타임 재시작 필수!**

In [ ]:
!pip install -q gradio huggingface_hub bitsandbytes

print("✅ 패키지 설치 완료")
print("")
print("⚠️  Colab 사용자:")
print("   런타임 재시작이 필요합니다!")
print("   Runtime → Restart runtime 클릭 후")
print("   Cell 4부터 다시 실행하세요")

## 1. 라이브러리 로드

In [ ]:
import os
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login

print(f"✅ Gradio 버전: {gr.__version__}")

## 3. 배포할 모델 설정

### 📝 학습한 모델들 목록

Day 1에서 학습한 모든 모델들을 배포할 수 있습니다:

| 노트북 | 모델 | HF 경로 (예시) |
|--------|------|----------------|
| 03번 | 한국어 요약 (EXAONE-3.5) | `ryanu/exaone-summary-lora` |
| 05번 | Granite 요약 | `ryanu/granite-summary-lora` |
| 05번 | Qwen3 요약 | `ryanu/qwen3-summary-lora` |
| 06번 | 감정 분류 (IMDB) | `ryanu/lora-sentiment` |
| 06번 | 영어 QA (SQuAD) | `ryanu/lora-qa` |

### 🚀 사용 방법

1. **모델 선택**: 아래 `MODELS`에서 배포하고 싶은 모델만 남기고 나머지는 주석 처리
2. **경로 변경**: `ryanu`를 본인 HF username으로 변경
3. **테스트**: Cell 7 실행하여 로컬에서 테스트

**기본값**: 한국어 요약 + 감정 분류 + 영어 QA (3개)

In [ ]:
# 여기를 수정하세요! 👇
# 배포하고 싶은 모델만 주석 해제하세요!

MODELS = {
    # ========================================
    # 03번: 한국어 요약 (EXAONE-3.5)
    # ========================================
    "한국어 요약 (03번)": {
        "base_model": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
        "lora_path": "ryanu/exaone-summary-lora",  # TODO: 본인 경로로 변경!
        "prompt_template": "{input}\n\n요약:",
        "max_new_tokens": 60,
        "placeholder": "뉴스 기사를 입력하세요...",
        "example": "서울시가 내년부터 전기차 충전소를 대폭 확대한다. 시는 2025년까지 공공 충전소 1만개를 설치할 계획이다.",
    },
    
    # ========================================
    # 05번: 다중 모델 비교 (선택)
    # ========================================
    # "Granite 요약 (05번)": {
    #     "base_model": "ibm-granite/granite-4.0-micro",
    #     "lora_path": "ryanu/granite-summary-lora",  # TODO: 본인 경로로 변경!
    #     "prompt_template": "<|user|>\n{input}\n\n위 기사를 요약해주세요.<|assistant|>\n",
    #     "max_new_tokens": 60,
    #     "placeholder": "뉴스 기사를 입력하세요...",
    #     "example": "서울시가 내년부터 전기차 충전소를 대폭 확대한다.",
    # },
    # "Qwen3 요약 (05번)": {
    #     "base_model": "Qwen/Qwen3-4B-Instruct-2507",
    #     "lora_path": "ryanu/qwen3-summary-lora",  # TODO: 본인 경로로 변경!
    #     "prompt_template": "<|im_start|>user\n{input}\n\n위 기사를 요약해주세요.<|im_end|>\n<|im_start|>assistant\n",
    #     "max_new_tokens": 60,
    #     "placeholder": "뉴스 기사를 입력하세요...",
    #     "example": "서울시가 내년부터 전기차 충전소를 대폭 확대한다.",
    # },
    
    # ========================================
    # 06번: 감정 분류 + 영어 QA
    # ========================================
    "감정 분류 (06번)": {
        "base_model": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
        "lora_path": "ryanu/lora-sentiment",  # TODO: 본인 경로로 변경!
        "prompt_template": "다음 영화 리뷰의 감정을 분류하세요.\n\n리뷰: {input}\n\n감정:",
        "max_new_tokens": 10,
        "placeholder": "영화 리뷰를 입력하세요...",
        "example": "This movie was amazing! Great story and excellent acting.",
    },
    "영어 QA (06번)": {
        "base_model": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
        "lora_path": "ryanu/lora-qa",  # TODO: 본인 경로로 변경!
        "prompt_template": "Context: The Eiffel Tower is in Paris, France.\n\nQuestion: {input}\n\nAnswer:",
        "max_new_tokens": 30,
        "placeholder": "질문을 입력하세요...",
        "example": "Where is the Eiffel Tower located?",
    },
}

print(f"✅ {len(MODELS)}개 모델 설정 완료")
for model_name in MODELS.keys():
    print(f"   - {model_name}")
    
print(f"\n💡 Tips:")
print(f"   - 'ryanu'를 본인 HF username으로 변경하세요!")
print(f"   - 05번 모델은 주석 해제하여 추가 가능")
print(f"   - 배포하고 싶은 모델만 선택하세요")

## 4. 모델 로드 함수

선택한 모델을 로드하는 함수입니다.

In [ ]:
# 전역 변수로 모델 캐싱
loaded_models = {}

def load_model(model_name):
    """모델을 로드하거나 캐시에서 가져오기"""
    if model_name in loaded_models:
        print(f"✅ {model_name} 캐시에서 로드")
        return loaded_models[model_name]
    
    config = MODELS[model_name]
    
    print(f"\n📥 {model_name} 로드 중...")
    
    # 토크나이저
    tokenizer = AutoTokenizer.from_pretrained(
        config["base_model"],
        use_fast=False
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Base 모델 (4-bit)
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    )
    
    base_model = AutoModelForCausalLM.from_pretrained(
        config["base_model"],
        device_map="auto",
        trust_remote_code=True,
        quantization_config=quant_config,
    )
    
    # LoRA 어댑터 로드
    model = PeftModel.from_pretrained(base_model, config["lora_path"])
    
    print(f"   ✅ {model_name} 로드 완료")
    
    # 캐시에 저장
    loaded_models[model_name] = (model, tokenizer, config)
    
    return model, tokenizer, config

print("✅ 모델 로드 함수 정의 완료")

## 5. 추론 함수

사용자 입력을 받아 모델로 추론하는 함수입니다.

In [ ]:
def generate_response(model_name, user_input):
    """선택한 모델로 추론 실행"""
    try:
        # 모델 로드
        model, tokenizer, config = load_model(model_name)
        
        # 프롬프트 생성
        prompt = config["prompt_template"].format(input=user_input)
        
        # 토큰화
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        # 생성
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=config["max_new_tokens"],
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        # 디코딩
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 프롬프트 제거
        if "요약:" in result:
            answer = result.split("요약:")[-1].strip()
        elif "감정:" in result:
            answer = result.split("감정:")[-1].strip()
        elif "Answer:" in result:
            answer = result.split("Answer:")[-1].strip()
        else:
            answer = result[len(prompt):].strip()
        
        return answer
    
    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"

print("✅ 추론 함수 정의 완료")

## 5. Gradio 인터페이스 생성

UI를 만듭니다.

In [ ]:
def create_demo():
    """Gradio 데모 생성"""
    
    with gr.Blocks(title="LoRA 모델 데모") as demo:
        gr.Markdown("""
        # 🤖 LoRA 파인튜닝 모델 데모
        
        Day 1에서 학습한 LoRA 모델을 테스트해보세요!
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                # 모델 선택
                model_dropdown = gr.Dropdown(
                    choices=list(MODELS.keys()),
                    value=list(MODELS.keys())[0],
                    label="📌 모델 선택",
                    interactive=True,
                )
                
                # 입력 텍스트
                input_text = gr.Textbox(
                    label="💬 입력",
                    placeholder=MODELS[list(MODELS.keys())[0]]["placeholder"],
                    lines=5,
                )
                
                # 버튼
                submit_btn = gr.Button("🚀 실행", variant="primary")
                clear_btn = gr.Button("🗑️ 초기화")
            
            with gr.Column(scale=1):
                # 출력
                output_text = gr.Textbox(
                    label="✨ 결과",
                    lines=10,
                    interactive=False,
                )
        
        # 예시
        gr.Markdown("### 📝 예시")
        example_boxes = []
        for model_name, config in MODELS.items():
            example_boxes.append(
                gr.Examples(
                    examples=[[config["example"]]],
                    inputs=[input_text],
                    label=f"{model_name} 예시",
                )
            )
        
        # 이벤트 핸들러
        def update_placeholder(model_name):
            return gr.update(placeholder=MODELS[model_name]["placeholder"])
        
        model_dropdown.change(
            fn=update_placeholder,
            inputs=[model_dropdown],
            outputs=[input_text],
        )
        
        submit_btn.click(
            fn=generate_response,
            inputs=[model_dropdown, input_text],
            outputs=[output_text],
        )
        
        clear_btn.click(
            fn=lambda: ("", ""),
            inputs=None,
            outputs=[input_text, output_text],
        )
    
    return demo

print("✅ create_demo 함수 정의 완료")

In [ ]:
demo = create_demo()

print("✅ Gradio 인터페이스 생성 완료")

# 미리 첫 번째 모델 로드 (빠른 응답을 위해)
print("\n📥 모델 미리 로드 중...")
first_model = list(MODELS.keys())[0]
try:
    load_model(first_model)
    print(f"   ✅ {first_model} 로드 완료!")
except Exception as e:
    print(f"   ⚠️ 로드 실패: {e}")
    print(f"   (첫 실행 시 다운로드됩니다)")

## 7. 로컬 테스트

아래 셀을 실행하면 로컬에서 UI를 테스트할 수 있습니다.

**Colab**: 자동으로 공개 링크 생성됨
**로컬**: http://localhost:7860 에서 확인

In [ ]:
# 로컬 테스트 실행

# ⚠️ 첫 실행 시 모델 다운로드로 시간이 걸릴 수 있습니다!
print("\n" + "="*60)
print("🚀 Gradio 데모 시작")
print("="*60)
print("\n💡 첫 실행 시:")
print("   - 모델 다운로드에 1-2분 소요")
print("   - '실행' 버튼 클릭 후 기다려주세요")
print("   - 로딩 중에는 타임아웃처럼 보일 수 있습니다")
print("\n" + "="*60 + "\n")

demo.launch(share=True, debug=True)  # debug=True로 오류 확인

## 8. HF Spaces 배포용 파일 생성

HuggingFace Spaces에 배포하려면 `app.py`와 `requirements.txt`가 필요합니다.

In [ ]:
import os

# 배포용 디렉토리 생성
deploy_dir = "./gradio_deploy"
os.makedirs(deploy_dir, exist_ok=True)

# app.py 생성
app_py_content = '''import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# 모델 설정 (여기를 수정하세요!)
MODELS = {
    # ========================================
    # 03번: 한국어 요약 (EXAONE-3.5)
    # ========================================
    "한국어 요약 (03번)": {
        "base_model": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
        "lora_path": "your-username/exaone-summary-lora",  # TODO: 본인 경로로!
        "prompt_template": "{input}\\n\\n요약:",
        "max_new_tokens": 60,
        "placeholder": "뉴스 기사를 입력하세요...",
        "example": "서울시가 내년부터 전기차 충전소를 대폭 확대한다.",
    },
    
    # ========================================
    # 05번: 다중 모델 비교 (선택사항)
    # ========================================
    # "Granite 요약 (05번)": {
    #     "base_model": "ibm-granite/granite-4.0-micro",
    #     "lora_path": "your-username/granite-summary-lora",
    #     "prompt_template": "<|user|>\\n{input}\\n\\n위 기사를 요약해주세요.<|assistant|>\\n",
    #     "max_new_tokens": 60,
    #     "placeholder": "뉴스 기사를 입력하세요...",
    #     "example": "서울시가 내년부터 전기차 충전소를 대폭 확대한다.",
    # },
    # "Qwen3 요약 (05번)": {
    #     "base_model": "Qwen/Qwen3-4B-Instruct-2507",
    #     "lora_path": "your-username/qwen3-summary-lora",
    #     "prompt_template": "<|im_start|>user\\n{input}\\n\\n위 기사를 요약해주세요.<|im_end|>\\n<|im_start|>assistant\\n",
    #     "max_new_tokens": 60,
    #     "placeholder": "뉴스 기사를 입력하세요...",
    #     "example": "서울시가 내년부터 전기차 충전소를 대폭 확대한다.",
    # },
    
    # ========================================
    # 06번: 감정 분류 + 영어 QA
    # ========================================
    "감정 분류 (06번)": {
        "base_model": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
        "lora_path": "your-username/lora-sentiment",
        "prompt_template": "다음 영화 리뷰의 감정을 분류하세요.\\n\\n리뷰: {input}\\n\\n감정:",
        "max_new_tokens": 10,
        "placeholder": "영화 리뷰를 입력하세요...",
        "example": "This movie was amazing! Great story and excellent acting.",
    },
    "영어 QA (06번)": {
        "base_model": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
        "lora_path": "your-username/lora-qa",
        "prompt_template": "Context: The Eiffel Tower is in Paris, France.\\n\\nQuestion: {input}\\n\\nAnswer:",
        "max_new_tokens": 30,
        "placeholder": "질문을 입력하세요...",
        "example": "Where is the Eiffel Tower located?",
    },
}

loaded_models = {}

def load_model(model_name):
    if model_name in loaded_models:
        return loaded_models[model_name]
    
    config = MODELS[model_name]
    
    tokenizer = AutoTokenizer.from_pretrained(config["base_model"], use_fast=False)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    
    base_model = AutoModelForCausalLM.from_pretrained(
        config["base_model"],
        device_map="auto",
        trust_remote_code=True,
        quantization_config=quant_config,
    )
    
    model = PeftModel.from_pretrained(base_model, config["lora_path"])
    
    loaded_models[model_name] = (model, tokenizer, config)
    return model, tokenizer, config

def generate_response(model_name, user_input):
    try:
        model, tokenizer, config = load_model(model_name)
        prompt = config["prompt_template"].format(input=user_input)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=config["max_new_tokens"],
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 프롬프트 제거
        if "요약:" in result:
            return result.split("요약:")[-1].strip()
        elif "감정:" in result:
            return result.split("감정:")[-1].strip()
        elif "Answer:" in result:
            return result.split("Answer:")[-1].strip()
        elif "<|assistant|>" in result:
            return result.split("<|assistant|>")[-1].strip()
        elif "<|im_start|>assistant" in result:
            return result.split("<|im_start|>assistant")[-1].replace("<|im_end|>", "").strip()
        else:
            return result[len(prompt):].strip()
    except Exception as e:
        return f"❌ 오류: {str(e)}"

with gr.Blocks(title="LoRA 모델 데모") as demo:
    gr.Markdown("# 🤖 LoRA 파인튜닝 모델 데모")
    gr.Markdown("Day 1에서 학습한 여러 LoRA 모델을 테스트해보세요!")
    
    with gr.Row():
        with gr.Column():
            model_dropdown = gr.Dropdown(
                choices=list(MODELS.keys()),
                value=list(MODELS.keys())[0],
                label="📌 모델 선택"
            )
            input_text = gr.Textbox(label="💬 입력", lines=5)
            submit_btn = gr.Button("🚀 실행", variant="primary")
        
        with gr.Column():
            output_text = gr.Textbox(label="✨ 결과", lines=10)
    
    submit_btn.click(
        fn=generate_response,
        inputs=[model_dropdown, input_text],
        outputs=[output_text]
    )

demo.launch()
'''

with open(f"{deploy_dir}/app.py", "w", encoding="utf-8") as f:
    f.write(app_py_content)

# requirements.txt 생성
requirements_content = '''gradio
transformers
torch
peft
bitsandbytes
accelerate
sentencepiece
'''

with open(f"{deploy_dir}/requirements.txt", "w") as f:
    f.write(requirements_content)

# README.md 생성
readme_content = '''---
title: LoRA Model Demo
emoji: 🤖
colorFrom: blue
colorTo: green
sdk: gradio
sdk_version: 4.0.0
app_file: app.py
pinned: false
---

# LoRA 파인튜닝 모델 데모

Day 1에서 학습한 LoRA 모델 데모입니다.

## 포함된 모델

- 한국어 요약 (03번) - EXAONE-3.5
- 감정 분류 (06번) - IMDB
- 영어 QA (06번) - SQuAD

## 선택 가능 (주석 해제)

- Granite 요약 (05번)
- Qwen3 요약 (05번)

## 사용 방법

1. 모델 선택
2. 입력 텍스트 입력
3. 실행 버튼 클릭
'''

with open(f"{deploy_dir}/README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)

print(f"\n✅ 배포용 파일 생성 완료!")
print(f"   📁 {deploy_dir}/")
print(f"      - app.py")
print(f"      - requirements.txt")
print(f"      - README.md")
print(f"\n💡 다음 단계:")
print(f"   1. app.py에서 'your-username'을 본인 HF username으로 변경")
print(f"   2. 배포하지 않을 모델은 주석 처리 또는 삭제")
print(f"   3. LoRA 어댑터를 HF Hub에 업로드")
print(f"   4. HF Spaces에 배포")
print(f"\n📊 기본 활성화:")
print(f"   - 한국어 요약 (03번)")
print(f"   - 감정 분류 (06번)")
print(f"   - 영어 QA (06번)")
print(f"\n   05번 모델은 주석 해제하여 추가 가능")

## 9. HF Spaces 배포 가이드

### 📋 배포 전 준비

1. **LoRA 어댑터 업로드** (05번 노트북 참고)
   ```python
   # 예시
   api.upload_folder(
       folder_path="./lora_summarization",
       repo_id="your-username/lora-summarization",
       repo_type="model"
   )
   ```

2. **app.py 수정**
   - `lora_path`를 HuggingFace 경로로 변경
   - 예: `"lora_path": "your-username/lora-summarization"`

### 🚀 배포 방법

**옵션 1: Web UI로 배포**
1. https://huggingface.co/spaces 접속
2. `New Space` 클릭
3. Space name 입력 (예: `lora-demo`)
4. SDK: `Gradio` 선택
5. `Create Space` 클릭
6. `gradio_deploy/` 폴더의 파일들 업로드:
   - `app.py`
   - `requirements.txt`
   - `README.md`

**옵션 2: CLI로 배포**
```bash
# HuggingFace CLI 설치
pip install huggingface_hub

# 로그인
huggingface-cli login

# Space 생성 및 업로드
cd gradio_deploy
git init
git add .
git commit -m "Initial commit"
git remote add origin https://huggingface.co/spaces/your-username/lora-demo
git push origin main
```

### ✅ 배포 확인

배포 후 약 1-2분 뒤:
- Space 페이지에서 "Running" 상태 확인
- 공개 URL로 접속 테스트
- 링크 공유 가능!

### 💰 비용

- **CPU 기반**: 무료 ✅
- **GPU 필요시**: PRO 계정 필요 ($9/월)

작은 모델(EXAONE-3.5-2.4B)은 CPU로도 충분합니다!

In [ ]:
# HF Hub CLI로 배포하기 (선택)
# 아래 주석을 해제하고 실행하세요

# from huggingface_hub import HfApi, login
# 
# # 로그인
# login(token="YOUR_HF_TOKEN")
# 
# api = HfApi()
# 
# # Space 생성
# api.create_repo(
#     repo_id="your-username/lora-demo",  # 본인 username으로 변경!
#     repo_type="space",
#     space_sdk="gradio"
# )
# 
# # 파일 업로드
# api.upload_folder(
#     folder_path="./gradio_deploy",
#     repo_id="your-username/lora-demo",
#     repo_type="space"
# )
# 
# print("✅ HF Spaces 배포 완료!")
# print("   링크: https://huggingface.co/spaces/your-username/lora-demo")

print("💡 위 주석을 해제하고 username을 수정한 후 실행하세요!")

## 10. 마무리 및 다음 단계

### 🎉 축하합니다!

Day 1의 모든 실습을 완료했습니다!

**배운 내용 정리:**
- ✅ 01: RAFT 데이터 전처리
- ✅ 02: 데이터 품질 검증
- ✅ 03: LoRA 파인튜닝
- ✅ 04: Baseline vs LoRA 비교
- ✅ 05: 다중 모델 + Chat Template 실험
- ✅ 06: 3개 태스크 비교 실험
- ✅ 07: Gradio 배포 + HF Spaces

### 💡 추가 실험 아이디어

1. **다른 태스크 시도**:
   - 번역, 코드 생성, 챗봇 등

2. **하이퍼파라미터 튜닝**:
   - LoRA rank, alpha, dropout 조합 실험

3. **앙상블**:
   - 여러 LoRA 모델 결합

4. **A/B 테스트**:
   - Gradio에서 두 모델 비교 UI 만들기

### 📚 참고 자료

- [Gradio 문서](https://www.gradio.app/docs)
- [HF Spaces 가이드](https://huggingface.co/docs/hub/spaces)
- [PEFT 문서](https://huggingface.co/docs/peft)

### 🚀 Day 2 예고

내일은 **RAG (Retrieval-Augmented Generation)**를 배웁니다!
- Naive RAG 구현
- Advanced RAG 기법
- Hybrid Search + Reranking

잘 쉬고 내일 만나요! 👋